In [1]:
import requests
from time import time
import pickle

# 1. Clustering covid topics

In [2]:
topicMap = dict()
topicMap['generic'] = ["corona", "#corona", "coronavirus", "#coronavirus", "covid", "#covid", "covid19", "#covid19", "covid-19", "#covid-19", "sarscov2", "#sarscov2", "sars cov2", "sars cov 2", "covid_19", "#covid_19", "pandemic", "#pandemic" ,"#coronaupdate"]
topicMap['covid_count'] = ["#ncov", "ncov", "#ncov2019", "ncov2019", "2019-ncov", "#2019-ncov", "#2019ncov", "2019ncov"]
topicMap['government_norms'] =  ["#lockdown", "lockdown"]
topicMap['safety_guidelines'] =  ["social distancing", "#socialdistancing", "wash ur hands", "wash your hands", "#washurhands", "#washyourhands" , "wearamask", "#wearamask", "wear a mask"]
topicMap['public_tackling_covid'] = ["quarantine", "#quarantine", "#selfisolating", "self isolating" , "flatten the curve", "flattening the curve", "#flatteningthecurve", "#flattenthecurve"]
topicMap['vaccines_and_medicines'] = ["vaccine", "vaccines", "#vaccine", "#vaccines", "corona vaccine", "corona vaccines", "#coronavaccine", "#coronavaccines"]
topicMap['herd_immunity'] = ["herd immunity", "#herdimmunity"]
topicMap['working_from_home'] = ["#stayathome", "#stayhome","#wfh", "work from home", "#workfromhome", "working from home", "#workingfromhome", "#stayhomestaysafe", "#hometasking"]
topicMap['protective_equipment'] = ["n95" , "ppe", "#ppe", "#n95", "hand sanitizer", "#handsanitizer" ,"#faceshields", "#masks4all", "face shield", "#faceshield", "face shields"] 
topicMap['people_not_following_measures'] = ["#covidiots", "covidiots"]
topicMap['comparison_with_flu']  = ["pneumonia", "#pneumonia"]
topicMap['covid_nick_names'] = ["chinese virus", "#chinesevirus", "wuhan virus", "#wuhanvirus", "kung flu", "#kungflu"]
topicMap['health_workers'] = ["health worker", "#healthworker", "health workers", "#healthworkers", "#frontlineheroes", "#coronawarriors"]
topicMap['home_schooling'] = ["#homeschool", "#homeschooling"]
                                    
pickle.dump(topicMap, open( "topics.pkl", "wb" ) )

# 2. Fetching Topic by Topic 

In [3]:
def build_data_dict(topic):
    data_dict = dict()
    query ='text:' + ','.join(topic)
    data_dict['q'] = query
    data_dict['rows'] = 1000
    return data_dict

In [4]:
import os
import json

start = time()

batch = 0

url = 'http://awesome-hw.sdsc.edu:8983/solr/political_data/select'

for topic in topicMap.keys():
    if topic == 'people_not_following_measures':
        for i in range(0,150000,1000):
            data_dict = build_data_dict(topicMap[topic])
            data_dict['wt'] = 'json'
            data_dict['start'] = i
            r = requests.post(url,data = data_dict)
            if not os.path.exists('political_tweets/' + topic):
                os.mkdir('political_tweets/' + topic)
            if eval(r.content)['response']['docs']:
                with open('political_tweets/' + topic +'/tweets_'+str(batch)+ '.json','wb') as f:
                    f.write(r.content)
            batch += 1 
            print("Completed batch Number %d"%batch)

end = time()

print('Time taken for entire batch rows: %f'%(end-start))

Completed batch Number 1
Completed batch Number 2
Completed batch Number 3
Completed batch Number 4
Completed batch Number 5
Completed batch Number 6
Completed batch Number 7
Completed batch Number 8
Completed batch Number 9
Completed batch Number 10
Completed batch Number 11
Completed batch Number 12
Completed batch Number 13
Completed batch Number 14
Completed batch Number 15
Completed batch Number 16
Completed batch Number 17
Completed batch Number 18
Completed batch Number 19
Completed batch Number 20
Completed batch Number 21
Completed batch Number 22
Completed batch Number 23
Completed batch Number 24
Completed batch Number 25
Completed batch Number 26
Completed batch Number 27
Completed batch Number 28
Completed batch Number 29
Completed batch Number 30
Completed batch Number 31
Completed batch Number 32
Completed batch Number 33
Completed batch Number 34
Completed batch Number 35
Completed batch Number 36
Completed batch Number 37
Completed batch Number 38
Completed batch Numbe

### For each topic create create labels and add emoji attribute

In [5]:
import pandas as pd
from glob import glob
import ipywidgets as widgets

#### Twitter setup

In [6]:
#!pip install tweepy
import os
import tweepy as tw
import text2emotion as te

##  Keys
consumer_key= '9DthIWnJQ7ZMzL2jEsBgPYsu4'
consumer_secret= 'BarxE5K10dENEYMIavMoYC2iG4OeGJKhz9KD8ymvjxHLvnOZYI'
access_token= '1196931775114514433-VDVfqYwRHgl5g6TdxYDnL2NjUum6GL'
access_token_secret= 'kiyWDHE7ItO6YLtlgsynu558kYeokYs0ShgcEudVGdTAq'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

[nltk_data] Downloading package stopwords to /home/akash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/akash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/akash/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
def parse_tweet_object(twobject, twdf, usrdf, pstdf, tgdf, mntndf, rtwtsdf, qtsdf, cntnsdf, rplsdf,topic):
    tid=twobject["id"]
    
    
    if "extended_tweet" in twobject.keys():
        twtext=twobject["extended_tweet"]["full_text"]
        thashtags=twobject["extended_tweet"]["entities"]["hashtags"]
        umentions=twobject["extended_tweet"]["entities"]["user_mentions"]
    else:
        twtext=twobject["text"]
        thashtags=twobject["entities"]["hashtags"]
        umentions=twobject["entities"]["user_mentions"]
    
    emotion_dict = fxget_emotionscore(twtext)        
    twdf.loc[tid,"happy"]= emotion_dict['Happy']
    twdf.loc[tid,"angry"]=emotion_dict['Angry']
    twdf.loc[tid,"surprise"]=emotion_dict['Surprise']
    twdf.loc[tid,"sad"]=emotion_dict['Sad']
    twdf.loc[tid,"fear"]=emotion_dict['Fear']
    twdf.loc[tid,"topic"]=topic
    
    if tid not in twdf.keys():
        twdf.loc[tid,"text"]=twtext
        twdf.loc[tid,"date"]=pd.to_datetime(twobject["created_at"])
        for key in replykeys:
            twdf.loc[tid,key] = twobject[key]
            
        #set these fields to false by default
        twdf.loc[tid,"retweet"] = False
        twdf.loc[tid,"quote"] = False
        twdf.loc[tid,"reply"] = False
    else:
        for key in replykeys:
            twdf.loc[tid,key] = max(twdf.loc[key], twobject[key])
   
    uid=twobject["user"]["id"]

    if uid not in usrdf.index:
        usrdf.loc[uid,"screen_name"]=twobject["user"]["screen_name"]
        usrdf.loc[uid,"created"]=pd.to_datetime(twobject["user"]["created_at"])
        usrdf.loc[uid,"description"]=twobject["user"]["description"]
        usrdf.loc[uid,"location"]=twobject["user"]["location"]

    #keep track of maximum number of friends found through database
    ufriends=twobject["user"]["friends_count"]
    
    if uid in usrdf.index:
        usrdf.loc[uid,"friends_count"] = \
            max(ufriends,usrdf.loc[uid,"friends_count"])
    else:
        usrdf.loc[uid,"friends_count"] = ufriends

    np=len(pstdf)
    pstdf.loc[np,"uID"] = uid
    pstdf.loc[np,"tID"] = tid

    #extract hashtags
    nh=len(tgdf)
    for p,ht in enumerate(thashtags):
        #twitter hashtags are not case sensitive, so they are converted to lowercase
        tgdf.loc[nh+p,"hashtag"] = ht["text"].lower()
        tgdf.loc[nh+p,"tID"] = tid

    #extract user mentions
    nu=len(mentionsdf)

    for p,ment in enumerate(umentions):
        mntndf.loc[nu+p,"tID"] = tid
        mntndf.loc[nu+p,"uID"] = ment["id"]
        
    #extract retweets
    if "retweeted_status" in twobject.keys():
        nr=len(rtwtsdf)
        rtwtsdf.loc[nr,"tID"] = tid
        tid_re=twobject["retweeted_status"]["id"]
        rtwtsdf.loc[nr,"tID_re"] = tid_re
        rtwtsdf.loc[nr,"topic"]=topic
        twdf.loc[tid,"retweet"] = True
    
    if "quoted_status" in twobject.keys():
        nq=len(qtsdf)
        qtsdf.loc[nq,"tID"] = tid
        qtsdf.loc[nq,"tID_qo"] = twobject["quoted_status"]["id"]
        qtsdf.loc[nq,"topic"]=topic
        twdf.loc[tid,"quote"] = True
 
    urls=twobject["entities"]["urls"]
    for url in urls:
        if "twitter.com/i/web/status/" not in url["expanded_url"]:
            nc=len(cntnsdf)
            cntnsdf.loc[nc,"url"] = url["url"]
            cntnsdf.loc[nc,"tid"] = tid
            cntnsdf.loc[nc,"expanded_url"] = url["expanded_url"]
            
        #else:
        #    tid_ln=int(url["expanded_url"].split("twitter.com/i/web/status/")[1])
        #    nl=len(linkedtweetsdf)
        #    linkedtweetsdf.loc[nl,"tid"] = tid
        #    linkedtweetsdf.loc[nl,"tid_ln"] = tid_ln
    
    #if "in_reply_to_status_id" in twobject.keys():
    if type(twobject["in_reply_to_status_id"]) != type(None):

        nl=len(rplsdf)
        rplsdf.loc[nl,"tID"] = tid
        rplsdf.loc[nl,"tID_rp"] = twobject["in_reply_to_status_id"]
        rplsdf.loc[nl,"uID_rp"] = twobject["in_reply_to_user_id"]
        rplsdf.loc[nl,"topic"]=topic
        twdf.loc[tid, "reply"] = True

In [23]:
def parse_all_tweets(jsonfile,tweetdf,userdf,postdf,mentionsdf, retweetsdf,quotesdf,tagsdf,containsdf,repliesdf,topic):
    tweet=json.load(open(jsonfile, "rb"))
    alltweets=tweet["response"]["docs"]
    ntweet=len(alltweets)
   
    for n in range(ntweet):
        atweet=alltweets[n]
        fulltweet=eval(atweet["fulltweets"])
        
        parse_tweet_object(fulltweet,
                           tweetdf, userdf, postdf, tagsdf, mentionsdf, 
                           retweetsdf, quotesdf, containsdf, repliesdf,topic)
        
        if "quoted_status" in fulltweet.keys():
            parse_tweet_object(fulltweet["quoted_status"],
                               tweetdf, userdf, postdf, tagsdf, mentionsdf, 
                               retweetsdf, quotesdf, containsdf, repliesdf,topic)
            
        if "retweet_status" in fulltweet.keys():
            parse_tweet_object(fulltweet["retweet_status"],
                               tweetdf, userdf, postdf, tagsdf, mentionsdf, 
                               retweetsdf, quotesdf, containsdf, repliesdf,topic)

In [37]:
replykeys=["reply_count", "retweet_count", "quote_count", "favorite_count"]

for i in range(1,len(all_topics) - 1):
    tweetfiles=glob("political_tweets/" + all_topics[i] + "/tweets_*.json")
    tweetdf=pd.DataFrame(columns=["text", "date","happy","angry","surprise","sad","fear","topic"] + replykeys + ["retweet", "quote", "reply"])
    userdf=pd.DataFrame(columns=["screen_name", "created","description",
                            "location", "friends_count"])
    #these may be reduntant as the hashtag and url serves as primary key
    #hashtags=pd.DataFrame(columns=["hashtag"])
    #url=pd.DataFrame(columns="url")
    
    #these relations will be edges in the graph
    postdf=pd.DataFrame(columns=["uID","tID","topic"])
    
    # tweets mentioning users
    mentionsdf=pd.DataFrame(columns=["tID","uID"])
    retweetsdf=pd.DataFrame(columns=["tID","tID_re","topic"])
    quotesdf=pd.DataFrame(columns=["tID", "tID_qo","topic"])
    tagsdf=pd.DataFrame(columns=["hashtag","tID"])
    containsdf=pd.DataFrame(columns=["url","tID", "expanded_url"])
    repliesdf=pd.DataFrame(columns=["tID","tID_rp", "uID_rp","topic"])
    
    #linkedtweetsdf=pd.DataFrame(columns=["tID","tID_ln"])
    #for file in tweetfiles:
    
    #parse_tweet_info(file)
    from ipywidgets import IntProgress
    from IPython.display import display

    wb = IntProgress(min=0, max=len(tweetfiles)) 
    display(wb)

    for n,file in enumerate(tweetfiles):
        pcent = float(n+1)/len(tweetfiles) * 100.
        print("\rReading %s ...  (%4.2f%%) " % (file, pcent), end="")
        parse_all_tweets(file,tweetdf,userdf,postdf,mentionsdf, retweetsdf,quotesdf,tagsdf,containsdf,repliesdf,all_topics[i])
        wb.value = n+1
        
    alldf={"tweets": tweetdf, "users": userdf, "posts": postdf, "mentions": mentionsdf, "retweets": retweetsdf,
      "quotes": quotesdf, "tags": tagsdf, "contains": containsdf,"replies":repliesdf}
    
    if not os.path.exists('political_tweets/pickles'):
        os.mkdir('political_tweets/pickles')
    pickle.dump(alldf, open("political_tweets/pickles/" + all_topics[i] + ".pkl", "wb"))
    
    print(" done.")

IntProgress(value=0, max=20)

Reading political_tweets/people_not_following_measures/tweets_6.json ...  (100.00%)  done.
